In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
%matplotlib tk

show_animation = True

In [21]:


class RRT:
    
    class Node:
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.path_x = []
            self.path_y = [] 
            self.parent = None
        
    
    def __init__(self, start, goal, obstacle_list, rand_area, delta_t = 1, max_iter = 1000):
        self.start = self.Node(start[0], start[1])
        self.goal = self.Node(goal[0], goal[1])
        self.max_iter = max_iter
        self.m_rand = rand_area[0]
        self.M_rand = rand_area[1]
        
        self.obstacle_list = obstacle_list
        self.rand_area = rand_area
        self.delta_t = delta_t
        
        self.tree = []
        
    def make_tree(self, animation = True):
        
        self.tree = [self.start]
        
        for i in range(self.max_iter):
            
            #random node 생성
            x_rand = self.Random_state()
            
            #가장 가까운 노드 찾기
            x_near = self.Nearest_neighbor(self.tree, x_rand)
            
            #새로운 노드 찾기
            x_new = self.New_state(x_near, x_rand, self.delta_t)
            
            #장애물과 충돌한다면 다시. 아니라면 트리에 추가.
            if not self.collision(x_new, self.obstacle_list):
                self.tree.append(x_new)
            else:
                continue
            
            if animation and i % 3 ==0:
                self.draw_map(x_rand)
                
            
            if self.Distance(self.tree[-1], self.goal) <= self.delta_t:
                final_node = self.New_state(self.tree[-1], self.goal, self.delta_t)
                if not self.collision(final_node, self.obstacle_list):
                    return self.make_course(len(self.tree) -1)
                
        return None
            
    
    def make_course(self, goal_ind):
        path = [[self.goal.x, self.goal.y]]
        node = self.tree[goal_ind]
        while node.parent is not None:
            path.append([node.x, node.y])
            node = node.parent
        path.append([node.x, node.y])
        return path
            
    def Distance(self, x_rand, x_):
        a = x_rand.x - x_.x
        b = x_rand.y - x_.y
        return math.sqrt(a**2 + b**2)
        
        
    def Random_state(self):
        
        if random.randint(0,100) >= 1 :
            x_rand = self.Node(random.uniform(self.m_rand, self.M_rand),
                               random.uniform(self.m_rand, self.M_rand))
        else:
            x_rand = self.Node(self.goal.x, self.goal.y)
        
        return x_rand
        
    
    def New_state(self, x_near, x_rand, delta_t):
        
        distance = self.Distance(x_rand, x_near)
        
        if distance <= delta_t:
            delta_t = distance
#             x_new = self.Node(x_rand.x, x_rand.y)
#             x_new.parant = x_near
#             return x_new
            
        x_new = self.Node( (x_rand.x - x_near.x)/distance + x_near.x,
                          (x_rand.y - x_near.y)/distance + x_near.y )
        
        x_new.parent = x_near
        
        return x_new
    
    
    
    @staticmethod
    def Nearest_neighbor(tree, x_rand):
        dist = [(x_rand.x - near.x) ** 2 + (x_rand.y - near.y) ** 2 for near in tree]
        x_near = tree[np.array(dist).argmin()]
        return x_near
    
    @staticmethod
    def collision(x_new, obstacle):
        x = x_new.x
        y = x_new.y
        
        for ox, oy, r in obstacle:
            if (x - ox) ** 2 + (y - oy) **2 <= r**2:
                return True
        
        return False
        
    
    def draw_map(self, rnd = None):
        plt.clf()

        if rnd is not None:
            plt.plot(rnd.x, rnd.y, "*k")

        for node in self.tree:
            if node.parent:
                plt.scatter(node.x,node.y, alpha = 0.3, color = 'b')
                plt.plot([node.x, node.parent.x], [node.y, node.parent.y], color = 'g')
#                 plt.plot(node.path_x, node.path_y, "-g")

        for (ox, oy, size) in self.obstacle_list:
            self.plot_circle(ox, oy, size)
        # 출발점과 도착점을 그려주고 
        plt.axis([-10, 20, -10, 20])
        plt.plot(self.start.x, self.start.y, "xr")
        plt.plot(self.goal.x, self.goal.y, "Dr")
        plt.grid(True)
        plt.pause(0.001)
    
    
    @staticmethod
    def plot_circle(x, y, size, color='-b'):
        deg = list(range(0,360,5))
        deg.append(0)
        xl = [x + size * math.cos(np.deg2rad(d)) for d in deg]
        yl = [y + size * math.sin(np.deg2rad(d)) for d in deg]
        plt.plot(xl, yl, color)
        
    

def main(gx=6.0, gy = 10.0):
    print("start")
    
    obstacleList = [
        (5, 5, 1),
        (3, 6, 2),
        (3, 8, 2),
        (3, 10, 2),
        (7, 5, 2),
        (9, 5, 2),
        (8, 10, 1)
    ]
    
    rrt = RRT(start=[0, 0],
              goal=[gx, gy],
              rand_area=[-7, 20],
              obstacle_list=obstacleList)
    path = rrt.make_tree(animation=show_animation)

    if path is None:
        print("Cannot find path")
    else:
        print("found path!!")

        # Draw final path
        if show_animation:
            rrt.draw_map()
#             plt.axis([-10, 20, -10, 20])
            plt.plot([x for (x, y) in path], [y for (x, y) in path], '-r')
            plt.grid(True)
            plt.pause(0.01)  # Need for Mac
            plt.show()


if __name__ == '__main__':
    main()


start
found path!!
